<a href="https://colab.research.google.com/github/MichelWars/exercicios-de-aulas/blob/main/aula_IA_22_03_25Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Instalação do pacote necessário
!pip install PyPDF2

# Importação de pacotes
import requests           # Usado para fazer requisições HTTP (baixar arquivos ou chamar APIs)
import PyPDF2             # Usado para ler arquivos PDF
import pickle             # Usado para salvar/ler dados em cache no formato binário
import os                 # Permite interagir com o sistema de arquivos (verificar se arquivos existem)
from io import BytesIO    # Permite manipular arquivos em memória (como se fossem arquivos físicos)




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.5 MB/s eta 0:00:00


In [5]:

# --- Configurações --- #
OPENROUTER_API_KEY = "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"  # Insira sua chave da OpenRouter
PDF_PATH = "/content/seuarquivo.pdf"  # Pode ser URL ou caminho local
CACHE_FILE = "pdf_cache.pkl"
MODEL_NAME = "mistralai/mistral-7b-instruct"



In [7]:

def carregar_pdf_com_cache():
    if os.path.exists(CACHE_FILE):
        print("Carregando texto do cache...")
        with open(CACHE_FILE, 'rb') as f:
            return pickle.load(f)

    print("Processando PDF pela primeira vez...")

    if PDF_PATH.startswith('http'):
        response = requests.get(PDF_PATH)
        arquivo_pdf = BytesIO(response.content)
    else:
        arquivo_pdf = open(PDF_PATH, 'rb')

    leitor = PyPDF2.PdfReader(arquivo_pdf)
    texto = "\n".join([pagina.extract_text() for pagina in leitor.pages])
    blocos = [texto[i:i+5000] for i in range(0, len(texto), 5000)]

    with open(CACHE_FILE, 'wb') as f:
        pickle.dump(blocos, f)

    return blocos



In [8]:

blocos_texto = carregar_pdf_com_cache()


Carregando texto do cache...


In [9]:

def perguntar_sobre_pdf(pergunta):
    contexto = blocos_texto[0][:10000]

    prompt = f"""
    [DOCUMENTO]:
    {contexto}

    [INSTRUÇÕES]:
    - Responda em português, baseado no documento acima.
    - Se não souber, diga "Não encontrado no PDF."

    [PERGUNTA]: {pergunta}
    """

    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://github.com/seu_usuario",
        "X-Title": "Assistente de PDF"
    }

    body = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "Você é um assistente que responde perguntas sobre documentos."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 500
    }

    response = requests.post(
        "https://openrouter.ai/api/v1/chat/completions",
        headers=headers,
        json=body
    )

    return response.json()["choices"][0]["message"]["content"]



In [10]:

while True:
    pergunta = input("Digite a pergunta , ou Sair: ")
    if 'Sair' not in pergunta:
        print(perguntar_sobre_pdf(pergunta))
    else:
        print('Sair')
        break



Digite a pergunta , ou Sair: Que horas são?
 A hora não é informada no PDF.
Digite a pergunta , ou Sair: Qual o modo de preparo?
 O modo de preparo do bolinho de cenoura, segundo o documento, é o seguinte:
1. Bater no liquidificador as cenouras, ovos e o óleo.
2. Transferir o conteúdo para a batedeira.
3. Adicionar o açúcar, farinha de trigo e por último o fermento.
4. Untar a assadeira e assar em forno pré-aquecido.
5. Para a calda, levar ao fogo a margarina e o achocolatado para derreter e engrossar.
6. Despejar sobre o bolo quente.


KeyboardInterrupt: Interrupted by user